# <span style="color:salmon">Astroquery</span> and its secrets
## Database access automation

**Pycoffe** (October 30, 2024)  
Carlos Cifuentes

Very often, it's all about data. Many tasks depend on retrieving information from large public databases that store observations, catalogues, and published studies. While manual downloads are sometimes possible, **automated data retrieval** is often the most efficient, secure, and reproducible approach. However, the **diversity of databases**, with different access protocols and formats, presents a challenge. ```astroquery``` simplifies this process, providing a Pythonic way to interact with numerous astronomical databases in a unified manner. This notebook will guide you through querying some of these databases. Many particular tweaks cannot be found in the official documentation.

In [1]:
import numpy as np
import pandas as pd
import astropy.coordinates as coord
import astropy.units as u
import re
import sys
#!{sys.executable} -m pip install termcolor
from termcolor import colored

## 1. SIMBAD

From the ```astroquery``` _library_, we import the ```astroquery.simbad``` _module_, specifically designed to access the SIMBAD database. Inside it, the ```Simbad``` _class_ provides different methods and syntaxes for performing searches and retrieving specific data about astronomical objects. All ```astroquery``` modules roughly follow the same API, which means that we can essentially use the same syntax for all of them.

In [2]:
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord 
from astropy import units as u

### Searching using coordinates

It is possible to search specifying coordinates and a search radius using the ```SkyCoord```class and the ```query_region()``` _method_:

In [3]:
result_table = Simbad.query_region(SkyCoord("03h46m24.2s +24d06m50s", frame='icrs'), radius=10*u.arcmin)
#result_table.pprint(show_unit=True, max_width=80, max_lines=5)
result_table

MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE,SCRIPT_NUMBER_ID
,"""h:m:s""","""d:m:s""",,,mas,mas,deg,,,,
object,str13,str13,int16,int16,float32,float32,int16,str1,str1,object,int32
Cl Melotte 22,03 46 24.2,+24 06 50,5,5,--,--,0,D,O,2021A&A...647A..19T,1
LAMOST J034622.46+240720.0,03 46 22.46,+24 07 20.0,6,6,1000.000,1000.000,90,D,O,2021RAA....21..292W,1
[KRL2001] 49,03 46 21.56,+24 07 17.0,6,6,--,--,0,D,,2001AJ....121..337K,1
V* PP Tau,03 46 31.1851,+24 07 02.431,14,14,0.112,0.093,90,A,O,2020yCat.1350....0G,1
Cl* Melotte 22 IPMBD 25,03 46 26.0867,+24 05 09.641,14,14,0.463,0.390,90,A,O,2020yCat.1350....0G,1
CXOP J034632.0+240746,03 46 32.01,+24 07 46.5,6,6,--,--,0,D,,2002ApJ...578..486D,1
[SCG94] X 135,03 46 29.7,+24 05 15,5,5,--,--,0,D,,,1
PGCC G166.44-23.59,03 46 24.280,+24 08 52.67,7,7,--,--,0,E,m,2016A&A...594A..28P,1


### Searching using names

Obtaining the catalogue name of an object or the equatorial coordinates is easy using SIMBAD with the ```query_objectids```method.

In [4]:
object_name = 'Cl Melotte 22'

result_table_id = Simbad.query_objectids(object_name)
result_table_id

ID
bytes19
[KC2019] Theia 369
C 0344+239
H 0346+24
M 45
NAME Pleiades
NAME Seven Sisters
OCl 421.0
[KPR2004b] 47
Cl Melotte 22


If we want to know the identification inside a particular catalogue, we can implement this module into a **function** that searches for a particular catalogue. For instance, the Gliese-Jahreis (GJ) identifier for a star would be:

In [5]:
def search_identifier(identifier, star_name):
    result_table = Simbad.query_objectids(star_name)
    matches = [row for row in result_table['ID'] if re.search(identifier, row)]
    if matches:
        for match in matches:
            print(colored(f"Found {identifier} identifier for {star_name}:", 'green'), f'{match}')
    else:
        print(colored(f"No {identifier} identifier for {star_name}", 'red'))

identifier = 'GJ'
star_name = 'Proxima Centauri'
search_identifier(identifier, star_name)

Found GJ identifier for Proxima Centauri: GJ 551


### Customizing SIMBAD query

Some parameters are not available by default. These is the default set:

In [6]:
result_table.colnames

['MAIN_ID',
 'RA',
 'DEC',
 'RA_PREC',
 'DEC_PREC',
 'COO_ERR_MAJA',
 'COO_ERR_MINA',
 'COO_ERR_ANGLE',
 'COO_QUAL',
 'COO_WAVELENGTH',
 'COO_BIBCODE',
 'SCRIPT_NUMBER_ID']

 We can create a customized SIMBAD using the ```add_votable_fields()``` method. The full list of parameters that are accesible in SIMBAD (i.e. VOTable) is:

In [7]:
Simbad.list_votable_fields()

--NOTES--

1. The parameter filtername must correspond to an existing filter. Filters include: B,V,R,I,J,K.  They are checked by SIMBAD but not astroquery.simbad

2. Fields beginning with rvz display the data as it is in the database. Fields beginning with rv force the display as a radial velocity. Fields beginning with z force the display as a redshift

3. For each measurement catalog, the VOTable contains all fields of the first measurement. When applicable, the first measurement is the mean one. 

Available VOTABLE fields:

bibcodelist(y1-y2)
biblio
cel
cl.g
coo(opt)
coo_bibcode
coo_err_angle
coo_err_maja
coo_err_mina
coo_qual
coo_wavelength
coordinates
dec(opt)
dec_prec
diameter
dim
dim_angle
dim_bibcode
dim_incl
dim_majaxis
dim_minaxis
dim_qual
dim_wavelength
dimensions
distance
distance_result
einstein
fe_h
flux(filtername)
flux_bibcode(filtername)
flux_error(filtername)
flux_name(filtername)
flux_qual(filtername)
flux_system(filtername)
flux_unit(filtername)
fluxdata(filtername)

A description of any field can be obtained using:

In [8]:
Simbad.get_field_description('fe_h')

Stellar parameters (Teff, log(g) and [Fe/H]) taken from the literature.


We may add as many fileds as needed. Let's find out some Pleiades members based on their astrometric data (distances and proper motions):

In [10]:
customSimbad = Simbad()
customSimbad.add_votable_fields('sptype', 'otype', 'fe_h', 'plx', 'pmra', 'pmdec')
result_table_custom = customSimbad.query_region(SkyCoord("03h46m24.2s +24d06m50s", frame='icrs'), radius=10*u.arcmin)
result_table_custom = result_table_custom[result_table_custom['PLX_VALUE'] > 7]
result_table_custom

MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE,SP_TYPE,SP_QUAL,SP_BIBCODE,OTYPE,Fe_H_Teff,Fe_H_log_g,Fe_H_Fe_H,Fe_H_flag,Fe_H_CompStar,Fe_H_CatNo,Fe_H_bibcode,PLX_VALUE,PMRA,PMDEC,SCRIPT_NUMBER_ID
,"""h:m:s""","""d:m:s""",,,mas,mas,deg,,,,,,,,unit-degK,cm/s**2,,,,,,mas,mas / yr,mas / yr,
object,str13,str13,int16,int16,float32,float32,int16,str1,str1,object,object,str1,object,object,int32,float32,float32,str1,object,str5,str19,float64,float64,float64,int32
Cl Melotte 22,03 46 24.2,+24 06 50,5,5,--,--,0,D,O,2021A&A...647A..19T,,,,OpenCluster,0,--,-0.0500,,SUN,,2024A&A...688A.165S,7.364,19.997,-45.548,1
V* PP Tau,03 46 31.1851,+24 07 02.431,14,14,0.112,0.093,90,A,O,2020yCat.1350....0G,M1.6,D,2020ApJ...892...31B,YSO,3995,4.6360,-0.0778,,,,2022ApJS..259...35A,7.393,18.380,-45.252,1
Cl* Melotte 22 IPMBD 25,03 46 26.0867,+24 05 09.641,14,14,0.463,0.390,90,A,O,2020yCat.1350....0G,,,,Low-Mass*,0,--,--,,,,,7.572,20.704,-44.848,1
V* OZ Tau,03 46 25.3897,+24 09 36.475,14,14,0.064,0.050,90,A,O,2020yCat.1350....0G,M3.1,D,2020ApJ...892...31B,Eruptive*,3562,5.4000,0.0000,,SUN,,2022MNRAS.514.4781X,7.191,16.921,-48.015,1
HD 23511,03 46 39.3335,+24 06 11.531,14,14,0.018,0.015,90,A,O,2020yCat.1350....0G,F5V,C,2001AJ....121.2148G,HighPM*,6630,--,--,,,,2021ApJ...921..117T,7.361,20.529,-46.942,1
V* V1189 Tau,03 46 12.8676,+24 03 15.754,14,14,0.015,0.013,90,A,O,2020yCat.1350....0G,M1.4,D,2020ApJ...892...31B,BYDraV*,3949,4.6400,-0.4500,,SUN,,2023ApJS..264...12H,7.332,22.068,-46.128,1
BD+23 518,03 46 07.7018,+24 03 41.819,14,14,0.015,0.012,90,A,O,2020yCat.1350....0G,GIV,D,,Star,0,--,--,,,,,7.817,-30.476,21.712,1
Gaia DR3 65230764998207232,03 46 16.0046,+24 11 22.765,14,14,0.039,0.025,90,A,O,2020yCat.1350....0G,,,,Star,0,--,--,,,,,7.395,20.756,-50.032,1


Because one _field_ can include different parameters, it is more convenient to map these fields into keywords that we can use in a function. For instance:

> ```main_id``` will map ```MAIN_ID```  
> ```ra``` will map ```RA```  
> ```dec``` will map ```DEC```  
> ...  
> ```teff```will map ```Fe_H_Teff```  
> ```logg```will map ```Fe_H_log_g```  
> ```fe_h```will map ```Fe_H_Fe_H```   

Mapping the parameters can be done using a dictionary:

In [11]:
def query_plus_simbad(object, data):

    customSimbad = Simbad()
    customSimbad.add_votable_fields('main_id', 'otype', 'otypes', 'sptype', 'sp_bibcode', 'dimensions',
                                     'plx', 'plx_error', 'plx_bibcode', 'distance', 'pmra', 'pm_err_maja',
                                     'pmdec', 'pm_err_mina', 'pm_bibcode', 'rvz_radvel', 'rvz_error',
                                     'rvz_bibcode', 'fe_h')
    
    result = customSimbad.query_object(object)

    data_mapping = {
        'pmra': 'PMRA',
        'pmra_err': 'PM_ERR_MAJA',
        'pmdec': 'PMDEC',
        'pmdec_err': 'PM_ERR_MINA',
        'pm_bibcode': 'PM_BIBCODE',
        'rv': 'RVZ_RADVEL',
        'rv_err': 'RVZ_ERROR',
        'rvz_bibcode': 'RVZ_BIBCODE',
        'distance': 'Distance_distance',
        'parallax': 'PLX_VALUE',
        'parallax_err': 'PLX_ERROR',
        'parallax_ref': 'PLX_BIBCODE',
        'object_type': 'OTYPE',
        'object_type_ext': 'OTYPES',
        'sp_type': 'SP_TYPE',
        'sp_ref': 'SP_BIBCODE',
        'teff': 'Fe_H_Teff'
    }
    return result[data_mapping[data]][0] if data in data_mapping else print('Item not found.')

object_name = 'Proxima Cen'
star_data = query_plus_simbad(object_name, 'teff')
star_data


2810

The result above is useless without its units. Luckily, the output is a masked column with some metadata, including units: ```<MaskedColumn name='Fe_H_Teff' dtype='int32' unit='unit-degK' description='Effective Temperature' length=1>```.

In [12]:
def query_plus_simbad(object_name):
    customSimbad = Simbad()
    customSimbad.add_votable_fields('main_id', 'otype', 'otypes', 'sptype', 'sp_bibcode', 'dimensions',
                                     'plx', 'plx_error', 'plx_bibcode', 'distance', 'pmra', 'pm_err_maja',
                                     'pmdec', 'pm_err_mina', 'pm_bibcode', 'rvz_radvel', 'rvz_error',
                                     'rvz_bibcode', 'fe_h')
    
    result = customSimbad.query_object(object_name)
    
    # Create a dictionary to hold data and units
    data_with_units = {}
    
    for column in result.colnames:
        value = result[column][0]  # Get the value
        unit = result[column].unit  # Get the unit
        if unit is None:
            unit = ''
        else:
            unit
        data_with_units[column] = {'value': value, 'unit': str(unit)}  # Store both in a dict

    return data_with_units

# Example usage
object_name = 'Proxima Cen'
star_data = query_plus_simbad(object_name)

if star_data is not None:
    for key, info in star_data.items():
        print(colored(f"{key}:", 'cyan'), f"{info['value']} {info['unit']}")


MAIN_ID: NAME Proxima Centauri 
RA: 14 29 42.9461 "h:m:s"
DEC: -62 40 46.164 "d:m:s"
RA_PREC: 14 
DEC_PREC: 14 
COO_ERR_MAJA: 0.024000000208616257 mas
COO_ERR_MINA: 0.03440000116825104 mas
COO_ERR_ANGLE: 90 deg
COO_QUAL: A 
COO_WAVELENGTH: O 
COO_BIBCODE: 2020yCat.1350....0G 
MAIN_ID_2: NAME Proxima Centauri 
OTYPE: Eruptive* 
OTYPES: Ro*|PM*|PM*|Er*|Ro*|MIR|PM*|**|V*|NIR|*|*|IR|UV|UV|X 
SP_TYPE: M5.5Ve 
SP_QUAL: C 
SP_BIBCODE: 1991AJ....101..662B 
SP_BIBCODE_2: 1991AJ....101..662B 
GALDIM_MAJAXIS: -- arcmin
GALDIM_MINAXIS: -- arcmin
GALDIM_ANGLE: 0 deg
GALDIM_QUAL:  
GALDIM_WAVELENGTH:  
GALDIM_BIBCODE:  
PLX_VALUE: 768.0665 mas
PLX_ERROR: 0.04989999905228615 mas
PLX_BIBCODE: 2020yCat.1350....0G 
Distance_distance: 1.3019 
Distance_Q:  
Distance_unit: pc 
Distance_merr: -5.0 
Distance_perr: -- 
Distance_method: paral 
Distance_bibcode: 2020yCat.1350....0G 
PMRA: -3781.741 mas / yr
PM_ERR_MAJA: 0.03099999949336052 mas / yr
PMDEC: 769.465 mas / yr
PM_ERR_MINA: 0.050999999046325684 mas /

### Accesing data

The output is stored in ```astropy.table.column``` format. There are several ways to access and manipulate column values.

In [13]:
object_name = 'Capella'
result_table = Simbad.query_object(object_name)
print(colored(f"\nStar: {object_name}", "green"))

# Array
ra_values_np = result_table['RA'].data
print(colored("RA as Numpy array:", 'blue'), ra_values_np)

# List
ra_values_list = result_table['RA'].tolist()
print(colored("RA as List:", 'blue'), ra_values_list)

# Row-by-row loop
for row in result_table:
    ra_value = row['RA']
    print(colored("RA row-by-row:", 'blue'), ra_value)

# Pandas' dataframe
df = result_table.to_pandas()
print(colored("RA row-by-row:", 'blue'), df['RA'][0])


Star: Capella
RA as Numpy array: ['05 16 41.3587']
RA as List: ['05 16 41.3587']
RA row-by-row: 05 16 41.3587
RA row-by-row: 05 16 41.3587


## VizieR

Analogously, from the ```astroquery``` _library_, we import the ```astroquery.vizier``` _module_, and the ```Vizier``` _class_ contained in it.

In [14]:
from astroquery.vizier import Vizier
import astropy.units as u

VizieR catalogues can be queried with the ```query_vizier``` or the ```query_region``` methods. These catalogues include all-sky surveys such as 2MASS, and also tabulated data from refereed publications:

In [ ]:
catalog_name = 'J/A+A/650/A201'
object_name = 'alf Cen'
result = Vizier.query_region(object_name, radius=5*u.arcsec, catalog=catalog_name)

print(colored('Title:', 'green'), Vizier(catalog=catalog_name).get_catalog_metadata()['title'][0]) 
print(colored('Authors:', 'green'), Vizier(catalog=catalog_name).get_catalog_metadata()['authors'][0]) 
result[0]

Any catalogue can be included and customized using a dictionary:

In [16]:
def query_vizier(name, catalog_id):
    catalog_dict = {
        'GALEX': ('II/312', ['FUVmag', 'e_FUVmag', 'NUVmag', 'e_NUVmag'], 'FUVmag'),
        'SDSS9': ('V/139/sdss9', ['umag', 'e_umag', 'gmag', 'e_gmag', 'rmag', 'e_rmag', 'imag', 'e_imag', 'zmag', 'e_zmag'], 'rmag'),
        '2MASS': ('II/246/out', ['RAJ2000', 'DEJ2000', 'Jmag', 'e_Jmag', 'Hmag', 'e_Hmag', 'Kmag', 'e_Kmag', 'Qflg'], 'Jmag'),
        'AllWISE': ('II/328/allwise', ['W1mag', 'e_W1mag', 'W2mag', 'e_W2mag', 'W3mag', 'e_W3mag', 'W4mag', 'e_W4mag', 'qph'], 'W1mag'),
        'Cifuentes20': ('J/A+A/642/A115', ['SpType', 'Lbol', 'e_Lbol', 'Radius', 'e_Radius', 'Mass', 'e_Mass', 'Multiple', 'Young', 'Excess'], 'SpType')
    }
    
    catalog_info = catalog_dict[catalog_id]
    catalog_id, columns, set_column = catalog_info
    
    v = Vizier(columns=columns, catalog=catalog_id)
    try:
        result = v.query_object(name)
        result = result[0][np.argmin(result[0][set_column])]
    except Exception:
        result = None
    
    return result

In [17]:
object_name = 'GJ 551'
star_data = query_vizier(object_name, 'Cifuentes20')

print(star_data)
print(colored(f'\n Stellar radius for {object_name}:', 'blue'), star_data['Radius'], '+/-', star_data['e_Radius'], '(Msol)')

SpType       Lbol            e_Lbol       Radius e_Radius   Mass   e_Mass Multiple Young Excess
            solLum           solLum       solRad  solRad  solMass solMass                      
------ ---------------- ---------------- ------- -------- ------- ------- -------- ----- ------
M5.5 V     0.0014422563     4.737831e-05  0.1504   0.0057  0.1347  0.0100    false false  false

 Stellar radius for GJ 551: 0.1504 +/- 0.0057 (Msol)


## 3. _Gaia_

In [18]:
from astroquery.gaia import Gaia

The _Gaia_ Archive uses the Astronomical Data Query Language (ADQL) to access its data. This is a preference given the flexibility and transparency that it provides. It needs a Gaia ID, which is available for 1.9 billion stars. We haver learnt that it is possible to obtain a Gaia DR3 identification number using ```astroquery```:

In [19]:
star_name = 'Proxima Cen'

def search_identifier(identifier, star_name):
    result_table = Simbad.query_objectids(star_name)
    matches = [row for row in result_table['ID'] if re.search(identifier, row)]
    if matches:
        for match in matches:
            return match.split()[-1]
    else:
        pass
        
def query_gaia(source_id):
    Gaia.login(user='ccifuent', password='nOtmY*pAsSw0rd')
    job = Gaia.launch_job("SELECT ra, dec, pmra, pmra_error, pmdec, pmdec_error, parallax, parallax_error, phot_g_mean_mag, l, b "
                          "FROM gaiadr3.gaia_source "
                          "WHERE source_id = {id}".format(id=source_id))
    r = job.get_results()
    return r

gaia_id = search_identifier('DR3', star_name)
print(colored('Gaia DR3:', 'green'), gaia_id, '\n')
gaia = query_gaia(gaia_id)
gaia

print(colored(f'\nDistance for {star_name}', 'green'), np.round(1000/gaia['parallax'][0], 4), 'parsec')


Gaia DR3: 5853498713190525696 

INFO: Login to gaia TAP server [astroquery.gaia.core]
OK
INFO: Login to gaia data server [astroquery.gaia.core]
OK

Distance for Proxima Cen 1.302 parsec


## 4. And more...

The full list of available services using ```astroquery``` can be found in [https://astroquery.readthedocs.io/en/latest/](https://astroquery.readthedocs.io/en/latest/).